本项目参考了squi2rel、MIOLOVERS等人的notebook，并成功支持deepdanbooru与xformers

其中deepdanbooru用于根据已有图片反推出对应的tag；而xformers经过实测，在kaggle平台上可以显著降低GPU显存，大幅提升速度，并且不影响出图质量(根据目前测试看来，在其他参数与GPU设备都相同的情况下，出的图是完全一致的)。**注意，不同的GPU设备，出的图是会有差异的。**

**不使用xformers，生成1280x960的图片(Euler, 28 steps, CFG Scale: 12):**

在T4与P100机器上，无论是txt2img还是img2img都需要1分20秒以上，显存爆满

**使用xformers，生成1280x960的图片(Euler, 28 steps, CFG Scale: 12):**

在T4机器上，txt2img需要40秒，img2img需要30秒，显存仅占用3~6G

在P100机器上，txt2img需要60秒，img2img需要50秒，显存仅占用3~6G

In [ ]:
# install webui
%cd /kaggle/working/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
    
# 选择一个可以正常使用的稳定版本commit
# 最新版目前看不到进度条https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/6774
%cd /kaggle/working/stable-diffusion-webui/
!git checkout 9ef41df6f9043d58fbbeea1f06be8e5c8622248b

In [ ]:
# link Novelai model
#!ln -s /kaggle/input/nai-pruned/animefull-final-pruned.ckpt /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/animefull-final-pruned.ckpt
#!ln -s /kaggle/input/nai-pruned/v1-inference.yaml           /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/animefull-final-pruned.yaml
#!ln -s /kaggle/input/nai-pruned/animevae.pt                 /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/animefull-final-pruned.vae.pt

# link anything-v3.0 model
#!ln -s /kaggle/input/anythingv30/Anything-V3.0.ckpt          /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.ckpt
#!ln -s /kaggle/input/anythingv30/Anything-V3.0.vae.pt        /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.vae.pt
#!ln -s /kaggle/input/anythingv30/Anything-V3.0-pruned.ckpt   /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0-pruned.ckpt

# link momoko model
!ln -s /kaggle/input/momoko/momoko-e.ckpt               /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/momoko-e.ckpt
!ln -s /kaggle/input/momoko/momoko-p.ckpt               /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/momoko-p.ckpt
!ln -s /kaggle/input/momoko/momoko-any3.0.ckpt          /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/momoko-any3.0.ckpt

# link 64-in-1 model
#!ln -s "/kaggle/input/64-in-1/64in1 推荐使用原版vae 这个可以按本模型0.9 3d模型0.1融合改善人物.ckpt" /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/64-in-1.ckpt

# download AbyssOrangeMix2 model
#!wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_sfw.safetensors -O /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/AbyssOrangeMix2_sfw.safetensors
#!wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -O /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/AbyssOrangeMix2_hard.safetensors
#!wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_nsfw.safetensors -O /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/AbyssOrangeMix2_nsfw.safetensors

# link hypernetwork
!ln -s /kaggle/input/nai-pruned/modules/         /kaggle/working/stable-diffusion-webui/models/hypernetworks

In [ ]:
# show gpu info
!nvidia-smi

# temporary fix for now, deepdanbooru error DNN library not found.
!apt install -y libcudnn8 --allow-change-held-packages

# install xformers
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

from subprocess import getoutput
s = getoutput('nvidia-smi')
if 'T4' in s:
    !pip install https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.14/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
elif 'P100' in s:
    !pip install https://github.com/Isotr0py/xformers-prebuild-wheels/raw/main/P100/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl

In [ ]:
# tag自动补全插件（可选）
%cd /kaggle/working/stable-diffusion-webui/
!git clone "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git" extensions/tag-autocomplete
%cd /kaggle/working/stable-diffusion-webui/extensions/tag-autocomplete
!git checkout 7fdad1bf623f1df0af72404afc1031d1e8adfa99

# 下载审美渐变插件（可选）
%cd /kaggle/working/stable-diffusion-webui/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients extensions/aesthetic-gradients
%cd /kaggle/working/stable-diffusion-webui/extensions/aesthetic-gradients
!git checkout 2624e5dd4902cb731287b426ad483e741ece85bb
    
# 下载历史记录/图像浏览器插件（可选）
%cd /kaggle/working/stable-diffusion-webui/
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser extensions/images-browser
%cd /kaggle/working/stable-diffusion-webui/extensions/images-browser
!git checkout a42c7a30181636a05815e62426d5eff4d3340529
    
# 中文UI界面翻译插件（可选）
%cd /kaggle/working/stable-diffusion-webui/
!git clone https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN extensions/localization-zh_CN
%cd /kaggle/working/stable-diffusion-webui/extensions/localization-zh_CN
!git checkout 7d106b8c180159d3826b70d351448e947efd6bdd

**Replicating NovelAI defaults**

It is possible to create outputs identical to NovelAI's current subscription service by doing the following:

**Standard (Euler)**

-Set the sampler to Euler (Not Euler A)

-Use 28 Steps

-Set CFG Scale to 11

-Use **masterpiece, best quality** at the beginning of all positive prompts

-Use **nsfw, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, artist name** as the negative prompt
(nsfw is optional, and toggled on the site)

-In the Settings tab, change **Stop last layers of CLIP model** to 2 and apply

**Euler-A Defaults**

If you want to replicate NovelAI Euler_a results, you must do the following alongside the defaults above:

-Under Sampler Parameters in settings, set '**ETA Noise Seed Delta**' to 31337

And you should be ready to prompt!

如果部署上有什么疑问可以参考教程：https://rentry.org/voldy

魔法咏唱：https://aitag.top/

元素法典：https://docs.qq.com/doc/DWEpNdERNbnBRZWNL

藏丹阁：https://docs.qq.com/doc/DY0lFeWZuVXRCdUJU

In [ ]:
#launch
%cd /kaggle/working/stable-diffusion-webui

# hacks to load model in gpu
!sed -i 's/map_location=None/map_location="cuda"/g' /kaggle/working/stable-diffusion-webui/modules/sd_models.py

# 使用最新版本的gradio时会有bug，无法正常工作，因此回退3.9版本
# 参考https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/6210
!sed -i 's/gradio==3.15.0/gradio==3.9.0/g' /kaggle/working/stable-diffusion-webui/requirements.txt

!COMMANDLINE_ARGS="--share --xformers --deepdanbooru --gradio-debug --lowram --disable-safe-unpickle --gradio-auth balls:balls,undefeatable:sonic" REQS_FILE="requirements.txt" python launch.py

In [ ]:
# 打包输出图片到/kaggle/working/outputs.tar内
# make output images into /kaggle/working/outputs.tar
%cd /kaggle/working/stable-diffusion-webui/
!tar -cvf outputs.tar ./outputs ./log
!mv ./outputs.tar ../

# 仅打包在webui中选择了保存的图片
!tar -cvf only_saved_img.tar ./log
!mv ./only_saved_img.tar ../

# 打包完后删除目录
!rm -rf ./outputs ./log
!mkdir ./outputs ./log